In [1]:
import pandas as pd
import numpy as np
#df_org=pd.read_csv(filepath+"application_train_merge_clear_nomissing .csv")
df_org=pd.read_csv("application_train_merge_clear_nomissing_with_ylabel_dy.csv")

In [2]:
df=df_org

In [22]:
df.head()

,Unnamed: 0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,simple_labelx,scores
0,0,1.0,0.0,202500.0,1.0,-9461.0,-637.0,-3648.0,-2120.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,96
1,1,0.0,0.0,270000.0,3.0,-16765.0,-1188.0,-1186.0,-291.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,100
2,2,0.0,0.0,67500.0,1.0,-19046.0,-225.0,-4260.0,-2531.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100
3,3,0.0,0.0,135000.0,1.0,-19005.0,-3039.0,-9833.0,-2437.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96
4,4,0.0,0.0,121500.0,1.0,-19932.0,-3038.0,-4311.0,-3458.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88


## 資料切割

In [3]:
#資料範例分割
from sklearn.model_selection import train_test_split
X , y = df.iloc[257511:,2:-3] , df.iloc[257511:,-2]
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.2,#超過10萬9,1分比較好 
                     random_state=0, 
                     stratify=y)
#資料正規化
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

## filter

In [12]:
corr_features = set()

# 建立相關矩陣
corr_matrix = X.corr()

for i in range(len(corr_matrix .columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.8:
            colname = corr_matrix.columns[i]
            corr_features.add(colname)
            
X.drop(labels=corr_features, axis=1, inplace=True)

In [13]:
corr_features = set()

# 建立相關矩陣
corr_matrix = X.corr(method='spearman')

for i in range(len(corr_matrix .columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.8:
            colname = corr_matrix.columns[i]
            corr_features.add(colname)
            
X.drop(labels=corr_features, axis=1, inplace=True)

### 方差特徵

In [14]:
from sklearn.feature_selection import VarianceThreshold
selector=VarianceThreshold(threshold=(0.8*(1-0.8)))
selector.fit_transform(X,y)
selector.get_support (indices= False )

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True, False, False,  True,  True, False,  True,
       False,  True,  True,  True, False,  True,  True,  True, False,
       False,  True, False,  True, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False,  True,  True])

In [15]:
from sklearn.feature_selection import SelectKBest,mutual_info_classif,f_classif
X_new = SelectKBest(f_classif, k=20).fit(X,y)

In [16]:
features=X.columns[X_new.get_support()]

In [17]:
features

Index(['CNT_CHILDREN', 'NAME_EDUCATION_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_ID_PUBLISH', 'FLAG_EMAIL', 'HOUR_APPR_PROCESS_START_x',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'NAME_CONTRACT_TYPE_Cash loans',
       'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Single / not married',
       'NAME_HOUSING_TYPE_Municipal apartment', 'OCCUPATION_TYPE_Laborers',
       'FONDKAPREMONT_MODE_reg oper account', 'HOUSETYPE_MODE_block of flats',
       'WALLSMATERIAL_MODE_Stone, brick'],
      dtype='object')

In [18]:
#資料範例分割
from sklearn.model_selection import train_test_split
X , y = df[features].iloc[257511:,:] , df.iloc[257511:,-2]
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.1,#超過10萬9,1分比較好 
                     random_state=0, 
                     stratify=y)
#資料正規化
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [19]:
X

,CNT_CHILDREN,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,FLAG_EMAIL,HOUR_APPR_PROCESS_START_x,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Single / not married,NAME_HOUSING_TYPE_Municipal apartment,OCCUPATION_TYPE_Laborers,FONDKAPREMONT_MODE_reg oper account,HOUSETYPE_MODE_block of flats,"WALLSMATERIAL_MODE_Stone, brick"
257511,2.0,3.0,-12784.0,-998.0,-4366.0,0.0,11.0,4.0,0.0,-3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
257512,2.0,3.0,-11915.0,-1887.0,-4420.0,0.0,12.0,7.0,0.0,-224.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
257513,0.0,1.0,-20259.0,-1224.0,-3455.0,0.0,6.0,0.0,0.0,-187.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
257514,0.0,1.0,-20650.0,-1224.0,-736.0,0.0,9.0,0.0,0.0,-549.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
257515,0.0,1.0,-19937.0,-2261.0,-3489.0,0.0,15.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.0,1.0,-9327.0,-236.0,-1982.0,0.0,15.0,0.0,0.0,-273.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
307507,0.0,1.0,-20775.0,-1224.0,-4090.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
307508,0.0,3.0,-14966.0,-7921.0,-5150.0,1.0,9.0,6.0,0.0,-1909.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
307509,0.0,1.0,-11961.0,-4786.0,-931.0,0.0,9.0,0.0,0.0,-322.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


## RFE

In [20]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
pipeline.fit(X,y)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))



Accuracy: 0.699 (0.006)


In [51]:
rfe.get_support()

array([False,  True, False,  True,  True,  True,  True, False, False,
       False, False,  True,  True, False, False, False, False, False,
        True, False,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

In [52]:
features_ref=rfe.get_support()

In [53]:
features_ref=X.columns[rfe.get_support()]

In [54]:
features_ref

Index(['AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
       'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START_x',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object')

In [6]:
from sklearn.neighbors import KNeighborsClassifier 

In [36]:
df_X = df[['DAYS_LAST_PHONE_CHANGE', 'DAYS_LATE_PAYMENT', 'PC_SK_DPDMix_Max',
       'SK_ID_PREV_y', 'bureau_DAYS_CREDIT']]
df_y = df['simple_labelx']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
307506    0.0
307507    0.0
307508    1.0
307509    0.0
307510    1.0
Name: simple_labelx, Length: 307511, dtype: float64

## RFECV

In [ ]:

from sklearn.feature_selection import RFECV
from sklearn.svm import SVR

estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=10)
selector = selector.fit(X, y)
selector.support_

selector.ranking_
